In [1]:
!pip install tensorflow_recommenders
!pip install googletrans
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=12307be250b0c330f1dd1e08e9620f7536e6f5d2ce2dbac4ab1874f7077d08f4
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstal

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load the data
jobreq_data = pd.read_csv("/content/cleaned_vacancies_data (1).csv")
cv_data = pd.read_csv("/content/data_train_final.csv")

In [ ]:
jobreq_data

In [ ]:
cv_data

Translate the datasets first to avoid biased points of similarities

In [12]:
from googletrans import Translator
from langdetect import detect
# Initialize the translator
translator = Translator()

# Function to detect and translate text
def translate_text(text):
    try:
        # Detect the language
        lang = detect(text)
        # Translate only if the text is in Indonesian
        if lang == 'id':
            translated = translator.translate(text, src='id', dest='en')
            return translated.text
        else:
            # Return the original text if it's not in Indonesian
            return text
    except Exception as e:
        # If detection or translation fails, return the original text
        return text
# Translate jobreq_data descriptions
jobreq_data['description'] = jobreq_data['description'].apply(translate_text)

# Translate cv_data text
cv_data['Text'] = cv_data['Text'].apply(translate_text)

calculate the similarity of the jobreq (job requirement) dataset and the cv's dataset

In [13]:
# Initialize the TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Concatenate all descriptions and texts to fit the vectorizer
all_texts = pd.concat([jobreq_data['description'], cv_data['Text']])
vectorizer.fit(all_texts)

# Transform the job descriptions and CV texts to their TF-IDF representation
jobreq_tfidf = vectorizer.transform(jobreq_data['description'])
cv_tfidf = vectorizer.transform(cv_data['Text'])

# Calculate cosine similarity using TensorFlow
def cosine_similarity_tf(tfidf_matrix1, tfidf_matrix2):
    tfidf_matrix1 = tf.constant(tfidf_matrix1.toarray(), dtype=tf.float32)
    tfidf_matrix2 = tf.constant(tfidf_matrix2.toarray(), dtype=tf.float32)
    dot_product = tf.linalg.matmul(tfidf_matrix1, tfidf_matrix2, transpose_b=True)
    norm_matrix1 = tf.sqrt(tf.reduce_sum(tf.square(tfidf_matrix1), axis=1, keepdims=True))
    norm_matrix2 = tf.sqrt(tf.reduce_sum(tf.square(tfidf_matrix2), axis=1, keepdims=True))
    norm_product = tf.linalg.matmul(norm_matrix1, norm_matrix2, transpose_b=True)
    similarity = dot_product / norm_product
    return similarity

In [14]:
# Calculate the similarity
similarity_matrix = cosine_similarity_tf(jobreq_tfidf, cv_tfidf)
similarity_matrix_np = similarity_matrix.numpy()

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix_np, index=jobreq_data.index, columns=cv_data.index)

In [15]:
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,392,393,394,395,396,397,398,399,400,401
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007233,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012319,0.000000,0.000000,0.000000,0.000000
1,0.010741,0.007278,0.000000,0.034775,0.018477,0.009317,0.003031,0.012212,0.021157,0.013530,...,0.019720,0.000000,0.003024,0.003294,0.002891,0.014938,0.006036,0.014573,0.030979,0.032173
2,0.007354,0.006912,0.000000,0.028211,0.017548,0.008849,0.002879,0.010368,0.020093,0.009255,...,0.016489,0.000000,0.002872,0.003129,0.002745,0.014187,0.005732,0.010580,0.022557,0.027733
3,0.037068,0.026243,0.010142,0.035401,0.030915,0.068930,0.038216,0.052265,0.039528,0.032645,...,0.042057,0.032600,0.004884,0.019574,0.011425,0.013084,0.014849,0.032086,0.062969,0.053590
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007233,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012319,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,0.048419,0.030885,0.005696,0.086029,0.035939,0.026799,0.046008,0.062352,0.038612,0.034117,...,0.073265,0.064298,0.008709,0.025461,0.029146,0.015555,0.031152,0.052658,0.072010,0.132063
493,0.000000,0.000000,0.009070,0.000000,0.000000,0.000000,0.000000,0.002650,0.017034,0.000000,...,0.000000,0.000000,0.012017,0.000000,0.000000,0.024623,0.005460,0.000000,0.004146,0.000000
494,0.032382,0.042460,0.004164,0.062803,0.020886,0.061615,0.044471,0.072169,0.034124,0.039182,...,0.074664,0.045045,0.012644,0.090071,0.036284,0.011272,0.018403,0.066758,0.095412,0.058210
495,0.015912,0.018912,0.007757,0.026927,0.021961,0.058625,0.021413,0.032037,0.031338,0.022060,...,0.047227,0.018961,0.001184,0.023468,0.016782,0.007407,0.019335,0.025275,0.089382,0.047674


Load the library for training the collaborative filtering model for candidate ranking

In [185]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dot, Flatten
from tensorflow.keras.models import Model

In [186]:
# Load the similarity matrix
similarity_matrix_np = similarity_matrix.numpy()

# Define function to prepare data for collaborative filtering
def prepare_data(similarity_matrix):
    # Get the indices of the top recommendations for each job
    top_candidates_indices = np.argsort(similarity_matrix, axis=1)[:, ::-1]
    # Create DataFrame to hold the user-item interactions
    interactions = []
    for job_idx, top_candidates in enumerate(top_candidates_indices):
        for candidate_idx in top_candidates:
            interactions.append([job_idx, candidate_idx, similarity_matrix[job_idx, candidate_idx]])
    cv_data_with_points = pd.DataFrame(interactions, columns=['job_idx', 'candidate_idx', 'points'])
    return cv_data_with_points

In [187]:
# Prepare data for collaborative filtering
cv_data_with_points = prepare_data(similarity_matrix_np)

# Convert indices to continuous IDs for embedding purposes
cv_data_with_points['job_idx'] = cv_data_with_points['job_idx'].astype('category').cat.codes
cv_data_with_points['candidate_idx'] = cv_data_with_points['candidate_idx'].astype('category').cat.codes

# Number of unique job requirements and candidates
n_jobs = cv_data_with_points['job_idx'].nunique()
n_candidates = cv_data_with_points['candidate_idx'].nunique()

# Define the input layers
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

# Define the embedding layers
user_embedding = Embedding(n_candidates, 20, embeddings_initializer='uniform', name='user_embedding')(user_input)
item_embedding = Embedding(n_jobs, 20, embeddings_initializer='uniform', name='item_embedding')(item_input)

# Flatten the embedding layers
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

# Compute the dot product of the embeddings
y = Dot(axes=1)([user_vecs, item_vecs])

# Create the model
model = Model(inputs=[user_input, item_input], outputs=y)
model.compile(optimizer='adam', loss='mse')

# Prepare the data for training
X = cv_data_with_points[['candidate_idx', 'job_idx']].values
y = cv_data_with_points['points'].values

# Train the model
model.fit([X[:, 0], X[:, 1]], y, epochs=128, batch_size=128, verbose=1)

Epoch 1/128
808/808 [==============================] - 2s 2ms/step - loss: 8.7582e-04
Epoch 2/128
808/808 [==============================] - 1s 2ms/step - loss: 3.4747e-04
Epoch 3/128
808/808 [==============================] - 1s 2ms/step - loss: 2.2516e-04
Epoch 4/128
808/808 [==============================] - 1s 2ms/step - loss: 1.8422e-04
Epoch 5/128
808/808 [==============================] - 1s 2ms/step - loss: 1.6766e-04
Epoch 6/128
808/808 [==============================] - 1s 2ms/step - loss: 1.5922e-04
Epoch 7/128
808/808 [==============================] - 1s 2ms/step - loss: 1.5514e-04
Epoch 8/128
808/808 [==============================] - 2s 2ms/step - loss: 1.5359e-04
Epoch 9/128
808/808 [==============================] - 2s 2ms/step - loss: 1.5229e-04
Epoch 10/128
808/808 [==============================] - 1s 2ms/step - loss: 1.5183e-04
Epoch 11/128
808/808 [==============================] - 2s 2ms/step - loss: 1.5172e-04
Epoch 12/128
808/808 [==============================

In [188]:
# Save the model to an .h5 file
model.save('collaborative_filtering_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from tensorflow.keras.models import load_model
# Load the saved model
model = load_model("/content/collaborative_filtering_model_final.h5")

Make a recommendation function to print the recommended candidate's cv

In [16]:
# Define a function to recommend CVs for a job requirement using the trained collaborative filtering model
def recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data, top_n=5):
    # Transform the job description to its TF-IDF representation
    jobreq_tfidf = vectorizer.transform([jobreq_data.loc[jobreq_id, 'description']])
    # Get the indices of CVs
    cv_indices = np.arange(len(cv_data))
    # Repeat the job requirement index for all CVs
    job_indices = np.repeat(jobreq_id, len(cv_data))
    # Predict the points using the model
    predicted_points = model.predict([cv_indices, job_indices])
    # Get the top recommended CVs
    top_cvs_indices = predicted_points.squeeze().argsort()[::-1][:top_n]
    # Get the CV texts and similarity scores
    top_cvs_with_scores = []
    for cv_idx in top_cvs_indices:
        cv_text = cv_data.loc[cv_idx, 'Text']
        similarity_score = predicted_points[cv_idx]
        top_cvs_with_scores.append((cv_text, similarity_score))
    return top_cvs_with_scores

In [ ]:
# Example: get top 5 CVs for a specific job requirement using the collaborative filtering model
jobreq_id = 5  # Replace with a job requirement index for testing

# you can print the code below to check the regarding jobreq_text
#jobreq_text = jobreq_data.iloc[jobreq_id]['description']
#print(jobreq_text)
top_cvs_with_scores = recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data)

print("Top CVs for Job Requirement ID {}: ".format(jobreq_id))
for cv_text, similarity_score in top_cvs_with_scores:
    print(f"CV: {cv_text}\nSimilarity Score: {similarity_score}\n")

In [18]:

import os
from PIL import Image
import numpy as np
import pickle
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm